<a href="https://colab.research.google.com/github/kuriatsu/learning_genetic_algorithms/blob/main/pulp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 84.6 MB/s eta 0:00:00


# 必要なモジュールのインポート
数理最適化アルゴリズムのパッケージとして、Pulpを使用
* 公式ドキュメント https://coin-or.github.io/pulp/  
* 日本語公式ブログ https://www.letsopt.com/entry/2020/08/12/232018  
* 今回の問題に近い？

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

# 入力データベース
DAY : 計画期間（30日間の配送拠点を計画する）
LIFT_NUM : リフトの数

リフトのデータベース  
* id : リフトID
* base : 拠点
* capacity : 燃料タンク容量[L]
* cosumption : 1日ごとののリフト消費量予測(リフト03 : 毎日3L消費)
* initial_remaining : 計画期間初日の燃料残量
* remaining : N日目の燃料残量、GA計算中にN日目の燃料残量をシミュレーションするために使用する変数

In [ ]:
DAY=30

lifts = [
  {
    "id":"03",
    "base": "kariya",
    "capacity": 60,
    "consumption": [3]*DAY,
    "initial_remaining": 60,
    "remaining": 60,
  },
  {
    "id":"30",
    "base": "takahama",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 40,
    "remaining": 40,
  },
  {
    "id":"52",
    "base": "takahama",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
  {
    "id":"19",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 30,
    "remaining": 30,
  },
  {
    "id":"32",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
]

 # Pulpでの定式化のための変数定義(データベースから抜き出しただけ)
課題を01整数計画問題に落とし込んでみた。

## 課題
* 給油時に燃料残量が回復する部分の定式化に失敗。線形問題ではなくなってしまうため、工夫が必要
* 目的関数のfuel_consumptionは燃料消費量ではなく、仮のもの。


In [ ]:
DATE = [d for d in range(0, DAY)]
LIFT_NUM=len(lifts)
LIFT = [l["id"] for l in lifts]

CONSUMPTION = {}
CAPACITY = {}
INITIAL_REMAINING = {}
BASE = {}

for lift in lifts:
  CONSUMPTION[lift["id"]] = lift["consumption"]
  CAPACITY[lift["id"]] = lift["capacity"]
  INITIAL_REMAINING[lift["id"]] = lift["initial_remaining"]

  if lift["base"] in BASE.keys():
    BASE[lift["base"]].append(lift["id"])
  else:
    BASE[lift["base"]] = [lift["id"]]

 # 定式化
 * 目的関数 objective : 給油拠点(`lpSum([delivery_base[d][b] for d in DATE for b in BASE.keys()])`と給油日数`lpSum(delivery_date)`の最小化
 * 変数
  * `remaining[lift][date]` : リフト`lift`の`date`日時点の燃料残量
  * `delivery_lift[lift][date]` : リフト`lift`へ`date`日に給油 (このリストを最適化する)
  * `delivery_base[date][base]` : `date`日に拠点`base`へ給油（目的関数用、delivery_liftから計算）
  * `delivery_day[date]` : `date`日に給油（目的関数用、delivery_liftから計算）
  

In [ ]:
## 変数定義

consumption = LpVariable.dicts("consumption", (LIFT, DATE), lowBound=0, cat="Continuous")
total_delivery = LpVariable.dicts("total_delivery", (LIFT, DATE), lowBound=0, cat="Continuous")
total_consumption = LpVariable.dicts("total_consumption", (LIFT, DATE), lowBound=0, cat="Continuous")
delivery_lift = LpVariable.dicts("delivery_lift", (LIFT, DATE), lowBound=0, cat="Binary")
delivery_base = LpVariable.dicts("delivery_base", (DATE, BASE.keys()), cat="Binary")
delivery_date = LpVariable.dicts("delivery_date", (DATE, BASE.keys()), cat="Binary")


## 目的関数
objective = lpSum([delivery_base[d][b] for d in DATE for b in BASE.keys()]) + lpSum(delivery_date)

## 目的関数登録 (最小化問題)
model = LpProblem("FuelConsumption", LpMinimize)
model += objective

## 制約条件
for lift in LIFT:
  ## 初日の燃料
  model += total_delivery[lift][0] == INITIAL_REMAINING[lift]
  model += total_consumption[lift][0] == 0
  ## 最終日の燃料残量の制約条件
  model += total_delivery[lift][DATE[-1]] - total_consumption[lift][DATE[-1]] - (CONSUMPTION[lift][DATE[-1]] + CONSUMPTION[lift][DATE[-2]]) >= 0

  for date in DATE[1:-2]:
    ## 燃料残量の推移 (次の日の燃料 = 前日の燃料 - 消費量 + 補給で満タン) <- 定式化失敗、２回目以降の給油でタンク容量をオーバーしてしまう可能性がある
    model += total_consumption[lift][date] == total_consumption[lift][date-1] + CONSUMPTION[lift][date]
    model += total_delivery[lift][date] == total_delivery[lift][date-1] + CAPACITY[lift] * delivery_lift[lift][date]
    ## 燃料残量の制約条件 (平均2日分の燃料消費量を下回らない)
    model += total_delivery[lift][date] - total_consumption[lift][date] - (CONSUMPTION[lift][DATE[date]] + CONSUMPTION[lift][DATE[date+1]]) >= 0
    ## 給油日
    model += delivery_date[date] == delivery_lift[lift][date]
    ## date日の給油拠点
    for base, lifts_in_base in BASE.items():
      if lift in lifts_in_base:
        model += delivery_base[date][base] == delivery_lift[lift][date]

## 最適化開始
sol = model.solve()
## 最適化結果、solver stateが Optimalで無い場合は、最適化失敗
print("solver state: ", LpStatus[sol])
print("optimized value:", value(model.objective))

solver state:  Optimal
optimized value: 4.0


# 結果表示
* delivery_liftにはどのリフトへ何日目に給油すべきかが格納
* remainingにはその日の残量(消費->給油後)
* delivery_dateには配送拠点

In [ ]:
## 結果可視化
for d in DATE:
  print(f"day:{d+1}")
  for l in LIFT:
    ## d日のリフトlが給油対象なら、liftsデータベースから対象のリフト情報を入手して表示
    if delivery_lift[l][d].value():
      for lift in lifts:
        if lift["id"] == l:
          print(f"-- BASE: {lift['base']}, LIFT: {lift['id']}, remaining: {value(total_delivery[lift['id']][d]) - value(total_consumption[lift['id']][d])}")

day:1
day:2
day:3
day:4
day:5
day:6
day:7
day:8
day:9
day:10
-- BASE: kariya, LIFT: 03, remaining: 93.0
-- BASE: takahama, LIFT: 30, remaining: 82.0
-- BASE: takahama, LIFT: 52, remaining: 61.0
-- BASE: higashiura, LIFT: 19, remaining: 72.0
-- BASE: higashiura, LIFT: 32, remaining: 61.0
day:11
day:12
day:13
day:14
day:15
day:16
day:17
day:18
day:19
day:20
day:21
day:22
day:23
day:24
day:25
day:26
day:27
day:28
day:29
day:30
